In [1]:
import scipy.io as sio
import datetime
import csv

In [11]:
# Formatted, unlabeled data. Formatted to be compatible with TrainSet tool. 
fDOM_fname = '../Data/data_annotating/non_formatted/fDOM_raw_10.1.12-7.13.20.txt' # 271008 entries
turb_fname = '../Data/data_annotating/non_formatted/turb_10.1.12-7.13.20.txt' # 
stage_fname = '../Data/data_annotating/non_formatted/stage_10.1.12-7.13.20.txt' # 272883
fDOM_cor_fname = '../Data/data_annotating/non_formatted/fDOM_corrected_10.1.12-7.13.20.txt'

In [12]:
# Read in and preprocess data: Convert timestamps to datetime objects and values to floats
with open(fDOM_fname, newline ='') as fDOM_file, open(turb_fname, newline ='') as turb_file, open(stage_fname, newline='') as stage_file:
    fDOM_reader = csv.reader(fDOM_file,delimiter = ',')
    stage_reader = csv.reader(stage_file,delimiter = ',')
    # Preprocess data to adjust entries that are one second off
    count = 0
    stage_data = []
    for row in stage_reader:
        try:
            time = datetime.datetime.strptime(row[0],'%d-%b-%Y %H:%M:%S')
        except: # Date was in a second format
            time = datetime.datetime.strptime(row[0],'%d-%b-%Y')
        if time.second != 0:
            time_delta = datetime.timedelta(seconds=1)
            time = time + time_delta
        if time.minute not in [0,15,30,45]:
            print('Minute: {} at i = {}'.format(time.minute, count))
        count +=1
        stage_data.append([time, ("%.17f" % float(row[1])).rstrip('0').rstrip('.')])

    count = 0 
    fDOM_data = []
    
    for row in fDOM_reader:
        try:
            time = datetime.datetime.strptime(row[0],'%d-%b-%Y %H:%M:%S')
        except: # Date was in a second format
            time = datetime.datetime.strptime(row[0],'%d-%b-%Y')
        if time.second != 0:
            time_delta = datetime.timedelta(seconds=1)
            time = time + time_delta
        if time.minute not in [0,15,30,45]:
            print('Minute: {} at i = {}'.format(time.minute, count))
        count +=1
        fDOM_data.append([time, ("%.17f" % float(row[1])).rstrip('0').rstrip('.')])
    
    turb_data = []
    prev_val = 0 # track previous value so that when an encoding error occurs we just use the previous value
    for i, line in enumerate(turb_file,1):
        vars = line.strip().split(',')
        # Correct encoding erros 
        if vars[1] == '\x00':
            vars[1] = prev_val
        # Correct times off by one second
        try:
            time = datetime.datetime.strptime(vars[0],'%d-%b-%Y %H:%M:%S')
        except: # Date was in a second format
            time = datetime.datetime.strptime(vars[0],'%d-%b-%Y')
        if time.second != 0:
            time_delta = datetime.timedelta(seconds=1)
            time = time + time_delta
        if time.minute not in [0,15,30,45]:
            pass
#             print('Minute: {} at i = {}'.format(time.minute, i))
        # Save data 
        try: 
            turb_data.append([time, ("%.17f" % float(vars[1])).rstrip('0').rstrip('.')])
        except ValueError:
            # print('i: {}, val: {} '.format(i,vars[1]))
            # print("Appending Prev Val: {}".format(prev_val))
            turb_data.append([time,float(prev_val)])
            continue # we don't want to assign prev_val to a faulty value
        prev_val = vars[1]
fDOM_file.close()
fDOM_c_file.close()
turb_file.close()
stage_file.close()

In [26]:
with open(fDOM_cor_fname, newline='') as fDOM_c_file:
    fDOM_cor_data = []
    fDOM_cor_reader = csv.reader(fDOM_c_file,delimiter = ',')
    prev_val = 0
    for row in fDOM_cor_reader:
        try:
            time = datetime.datetime.strptime(row[0],'%d-%b-%Y %H:%M:%S')
        except: # Date was in a second format
            time = datetime.datetime.strptime(row[0],'%d-%b-%Y')
        if time.second != 0:
            time_delta = datetime.timedelta(seconds=1)
            time = time + time_delta
        if time.minute not in [0,15,30,45]:
#             print('Minute: {} at i = {}'.format(time.minute, count))
            pass
#             continue
        try: 
            fDOM_cor_data.append([time, ("%.17f" % float(row[1])).rstrip('0').rstrip('.')])
            prev_val = row[1]
        except ValueError:
            fDOM_cor_data.append([time, ("%.17f" % float(prev_val)).rstrip('0').rstrip('.')])
            

In [34]:
print(len(fDOM_cor_data))
print(len(fDOM_data))
print(len(fDOM_corrected_filtered))
print(fDOM_corrected_filtered[-1])

276997
271008
[datetime.datetime(2015, 6, 16, 11, 0), '45.43619999999999948']


In [35]:
# Align entries of all timeseries by time. 
delta = datetime.timedelta(minutes=15)

time_corrected_fDOM = []
# fDOM has large swaths of missing data: only add stage and turb that has matching dates
stage_filtered = []
turb_filtered = []
fDOM_corrected_filtered = []
for i in range(len(fDOM_data)):
    fDOM_time = fDOM_data[i][0]
    j = i
    while fDOM_time != stage_data[j][0]:
        j+=1
    stage_filtered.append(stage_data[j])
    j = i
    while fDOM_time != turb_data[j][0]:
        j+=1
    turb_filtered.append(turb_data[j])
    j = i
    while fDOM_time != fDOM_cor_data[j][0]:
        j+=1
    fDOM_corrected_filtered.append(fDOM_cor_data[j])
    if not(fDOM_time == stage_filtered[i][0] == turb_filtered[i][0]):
        print('Error: ',i)
print('{}. {}. {} '.format(fDOM_data[100], stage_filtered[100], turb_filtered[100]))

IndexError: list index out of range

In [18]:
print('{}. {}. {} '.format(fDOM_data[260000], stage_filtered[260000], turb_filtered[260000], fDOM_))
print(len(stage_filtered))

[datetime.datetime(2020, 3, 20, 20, 45), 35.98333]. [datetime.datetime(2020, 3, 20, 20, 45), 0.5177273]. [datetime.datetime(2020, 3, 20, 20, 45), -4.500798] 
271008


In [5]:
stage_data = stage_filtered
turb_data = turb_filtered
fDOM_cor_data = fDOM_corrected_filtered

In [6]:
delta = datetime.timedelta(minutes=15)
lower_limit = 100000 # limit number of entries written out. TrainSet can't quite handle the all 270k records
upper_limit = 200000
write_fname = '../Data/data_anotating/non_annotated_csv/timeseriesCorr_0-100000k.csv'
with open(write_fname,'w',newline='') as outfile:
    writer = csv.writer(outfile,delimiter=',')
    writer.writerow(['series', 'timestamp', 'value', 'label'])
    for i in range(len(fDOM_data)):
        if i <= lower_limit or i>=upper_limit:
            break 
        fDOM_time = fDOM_data[i][0] 
        stage_time = stage_data[i][0]
        turb_time = turb_data[i][0]
        fDOM_cor_time = fDOM_cor_data[i][0]
        # Make sure that all timestamps are the same
        if not(fDOM_time == stage_time == turb_time):
            print('Error: {}'.format(i))
            print('fDOM: {}'.format(fDOM_time))
            print('Stage: {}'.format(stage_time))
            print('Turb: {}'.format(turb_time))
        
        # Append in specific order 
        fDOM_time = fDOM_time.isoformat() + '.000Z'
        stage_time = stage_time.isoformat() + '.000Z'
        turb_time = turb_time.isoformat() + '.000Z'
        fDOM_cor_time = fDOM_cor_time.isoformat() + '.000Z'
        
        writer.writerow(['fDOM', fDOM_time, fDOM_data[i][1],''])
        writer.writerow(['Stage', stage_time, stage_data[i][1],''])
        writer.writerow(['Turbidity',turb_time, turb_data[i][1], ''])
        writer.writerow(['fDOM_Cor', fDOM_cor_time, fDOM_cor_data[i][1], ''])
outfile.close()

In [8]:
# Observe where data in missing in fDOM
for i in range(len(fDOM_data)):
    curr = fDOM_data[i][0]
    prev = fDOM_data[i-1][0]
    if prev + delta != curr:
        print("i: {} Curr: {}  Prev: {}".format(i,curr,prev))

i: 0 Curr: 2012-10-01 03:45:00  Prev: 2020-07-13 12:30:00
i: 6848 Curr: 2012-12-11 12:00:00  Prev: 2012-12-11 11:30:00
i: 55908 Curr: 2014-05-06 13:45:00  Prev: 2014-05-06 12:45:00
i: 88646 Curr: 2015-04-12 14:30:00  Prev: 2015-04-12 14:00:00
i: 99053 Curr: 2015-08-12 00:00:00  Prev: 2015-07-30 00:00:00
i: 105153 Curr: 2015-10-14 13:45:00  Prev: 2015-10-14 12:45:00
i: 105601 Curr: 2015-10-19 13:45:00  Prev: 2015-10-19 05:30:00
i: 131240 Curr: 2016-07-12 15:45:00  Prev: 2016-07-12 15:15:00
i: 164929 Curr: 2017-06-28 14:15:00  Prev: 2017-06-28 13:45:00
i: 165701 Curr: 2017-07-06 15:30:00  Prev: 2017-07-06 15:00:00
i: 166458 Curr: 2017-07-14 13:00:00  Prev: 2017-07-14 12:30:00
i: 168254 Curr: 2017-08-02 11:45:00  Prev: 2017-08-02 05:45:00
i: 175142 Curr: 2017-10-13 16:00:00  Prev: 2017-10-13 05:30:00
i: 175406 Curr: 2017-10-21 15:30:00  Prev: 2017-10-16 09:45:00
i: 177595 Curr: 2017-11-13 11:00:00  Prev: 2017-11-13 10:30:00
i: 191331 Curr: 2018-04-05 13:15:00  Prev: 2018-04-05 12:45:00
i: